In [1]:
#! python3

"""
A python script that takes a directory continging Mason Petrology reports as pdf's and uses TesseractOCR and openCV to automatically extract the sample numbers, minerals and mineral modes for loading into SA_Geodata
"""

"\nA python script that takes a directory continging Mason Petrology reports as pdf's and uses TesseractOCR and openCV to automatically extract the sample numbers, minerals and mineral modes for loading into SA_Geodata\n"

In [39]:
#Library imports
import os
from pdf2image import convert_from_path, convert_from_bytes
import tempfile
import shutil
import re
import cv2 
import numpy as np
import pytesseract
from pytesseract import Output
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from timeit import default_timer as timer
import json
from datetime import datetime


In [3]:
start = timer()
#scan a file directory and return a list of pdf files to iterate over

#pdf_path = os.path.normpath(str(input(f'Please input the full path to the pdf files:')))
pdf_path = 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext'


pdf_list = []
with os.scandir(pdf_path) as it:
    for entry in it:
        if entry.name.endswith(".pdf") and entry.is_file():
            pdf_list.append(entry.path)

In [4]:

print(pdf_list)  # Print statement for testing

['C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3286.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3294.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3347.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3363.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3792.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3807.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3997.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3998.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4065.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4069.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4109.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4114.pdf', 'C:\\Users\\dutchr01\\MyPyS

In [5]:
#create a temp directory and Convert pdf pages to individual jpg images for OCR and image extraction
jpg_path_list = []
temp_dir = tempfile.TemporaryDirectory()

for pdf in pdf_list:
    images = convert_from_path(pdf, dpi = 300, output_folder=temp_dir.name,fmt='jpg')
    
# close PIL image objects
for im in images:
    im.close()

#scan a file directory and return a list of jpg files to iterate over
#with os.scandir(temp_dir.name) as it:
    #for entry in it:
        #if entry.name.endswith(".jpg") and entry.is_file():
            #jpg_path_list.append(entry.path)


In [6]:


# func to detect vertical and horizontal lines from image
def lines(im, x=50): # x number between 1 and 255 (default 50)
    # Defining a kernel length
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(im).shape[1]//x

    # A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    # A kernel of (3 X 3) ones.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # Morphological operation to detect vertical lines from an image
    img_temp1 = cv2.erode(binary_image, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    #cv2.imwrite("verticle_lines.jpg",verticle_lines_img)
    # Morphological operation to detect horizontal lines from an image
    img_temp2 = cv2.erode(binary_image, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

#func to sort the found contours
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [7]:
#print(jpg_path_list)  #print statement for testing

In [8]:
with os.scandir(temp_dir.name) as it:
    for idx,im_path in enumerate(it):
        print(im_path.path)

c-4058-85cd-53675db533a0-09.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-10.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-11.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-12.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-13.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-14.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-15.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-16.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-17.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-18.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-19.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-20.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-21.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-22.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-23.jpg
C:\Temp\tmp64lfaz_y\db4c2128-a86c-4058-85cd-53675db533a0-24.jpg
C:\Temp\

In [9]:
# config for TesseractOCR
custom_config = r'--oem 3 --psm 6'

mins_dict = {'R_No':[],'Mineral':[],'Vol %':[]}

#RegEx patterns for sample nos and volume% charachters
sample_no_pattern = '^(\d{7})$|^(R\d{7})$'
Vol_pattern ='^((Tr)|(<1)|[\d]|\d{2})$' 

# for loop that iterates over jpg files in temp file, opens image and runs TesseractOCR on them
with os.scandir(temp_dir.name) as it:
    for idx,im_path in enumerate(it):
        if im_path.name.endswith(".jpg") and im_path.is_file():
            image = cv2.imread(im_path.path, 0)
            d = pytesseract.image_to_data(image, config=custom_config,output_type=Output.DICT)
            n_boxes = len(d['text'])
            
            # nested for loop that selects images that have the first line/first word == to 'SAMPLE'
            for i in range(n_boxes):
                if int(d['line_num'][i]) == 1 and int(d['word_num'][i]) == 1 and d['text'][i] =='SAMPLE':
                                
                    #nested for loop to use regex to grab the sample number from the line 1 of the selected jpg pages
                    for i in range(n_boxes):
                        if int(d['line_num'][i]) == 1:
                            if re.match(sample_no_pattern, d['text'][i]):
                                sample_no = (d['text'][i]).strip('R')
                                #sample_nos.append(d['text'][i])

                    ### section to find boxed mineralogy part of image 
                    thresh, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

                    #call lines function to find lines in image
                    lines(image,50)

                    # Weighting parameters, this will decide the quantity of an image to be added to make a new image.
                    alpha = 0.5
                    beta = 1.0 - alpha
                    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
                    img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
                    img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
                    (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

                    # Find contours for image, which will detect all the boxes
                    contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                    # Sort all the contours by top to bottom.
                    (contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

                    for c in contours:
                        # Returns the location and width,height for every contour
                        x, y, w, h = cv2.boundingRect(c)
                        new_img = image[y:y+h,x:x+w]
                    
                    text = pytesseract.image_to_data(new_img,  config=custom_config, output_type=Output.DICT)

                    x_boxes = len(text['text'])

                    mins_lst = []
                    vol_lst = []
                    sample_lst = []
                    
                    for i in range(x_boxes):
                        #if int(text['conf'][i]) > 30:
                        # if int(text['line_num'][i]) > 1:
                        if int(text['word_num'][i])==1 and not text['text'][i] == 'Mineral':
                            mins_lst.append((text['text'][i]))

                                #if int(text['word_num'][i])==1:
                                    

                    for i in range(x_boxes):
                        #if int(text['conf'][i]) > 30:
                            #if int(text['line_num'][i]) > 1:
                        if int(text['left'][i]) < 1250:
                            if re.match(Vol_pattern, text['text'][i]):
                                vol_lst.append((text['text'][i]))

                    for i in range(len(mins_lst)):
                        sample_lst.append(sample_no)

                    mins_dict['Mineral'].append(mins_lst)
                    mins_dict['Vol %'].append(vol_lst)
                    mins_dict['R_No'].append(sample_lst)

                
        else:
            pass

In [10]:
# remove temp directory
shutil.rmtree(temp_dir.name)
end = timer()

In [11]:
# print statments to indicate all pages OCR'd successfully
Number_samples = len(mins_dict['R_No'])
No_min_lists = len(mins_dict['Mineral'])
No_vol_lists = len(mins_dict['Vol %'])
print(f'It took {(end-start)} seconds to process a total of:')
print(f'{Number_samples} samples')
print(f'With {No_min_lists} mineral lists')
print(f'And {No_vol_lists} volume lists')

It took 4802.613309234999 seconds to process a total of:
552 samples
With 552 mineral lists
And 552 volume lists


In [30]:
# Generate a pandas df of the mins_dict of sample lists
df = pd.DataFrame(mins_dict)

In [31]:
# Select completed records and explode the lists to individual values and select falied records for out put

df_complete = df[df['Mineral'].str.len() == df['Vol %'].str.len()].apply(pd.Series.explode)
df_failed = df[df['Mineral'].str.len() != df['Vol %'].str.len()]


In [32]:
#Load mineral code mapping dict
min_dict = json.load(open('C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\min_dict.json'))

In [33]:
#ToDo strip R's from sample no column

In [34]:
# Convert Vol % column from str to int and replace str values (like Tr or <1) to 0
df_complete['mode'] = pd.to_numeric(df_complete['Vol %'],errors = 'coerce').replace(np.nan, 0, regex=True).astype(int)

In [35]:
# Make a new col and populate with SA_Geo min abndance codes based on conditional args from the converted Vol %s
df_complete['SA_Geo_mode'] = str(np.nan)
df_complete.loc[df_complete['mode']==0,'SA_Geo_mode'] = 'RARE'
df_complete.loc[df_complete['mode'] >= 70,'SA_Geo_mode'] = 'ABUNDANT'
df_complete.loc[(df_complete['mode'] >= 30) & (df_complete['mode'] < 70),'SA_Geo_mode'] = 'MAJOR'
df_complete.loc[(df_complete['mode'] >= 5) & (df_complete['mode'] < 30),'SA_Geo_mode'] = 'MINOR'
df_complete.loc[(df_complete['mode'] > 0) & (df_complete['mode'] < 5),'SA_Geo_mode'] = 'TRACE'

In [36]:
# Run a check that the Mins in the Mineral col are in the mineral map JSON dict keys and output a missing_mins df for missing keys
df_complete['check']=df_complete['Mineral'].str.strip('/?, ').isin(min_dict.keys())
missing_mins = df_complete[df_complete['check']==False]
missing_mins

,R_No,Mineral,Vol %,mode,SA_Geo_mode,check
5,2430062,Voids,20,20,MINOR,False
7,2427076,Mesostasis,7,7,MINOR,False
8,2430064,Mesostasis,40,40,MAJOR,False
9,2430067,Mesostasis,5,5,MINOR,False
11,2430070,Mesostasis,35,35,MAJOR,False
...,...,...,...,...,...,...
487,1963547,Leucoxeneftitanite,1,1,TRACE,False
489,1969096,Mesostasis,5,5,MINOR,False
490,1969097,Mesostasis,3,3,TRACE,False
524,1842270,Opaline,<1,0,RARE,False


In [37]:
# Create a new Col and populate with the mapped SA_Geo min codes, replacing Mins not found in the mineral map JSON dict with the 'UKN' mineral code
df_complete['SA_geo_min'] = str(np.nan)
df_complete['SA_geo_min'] = df_complete['Mineral'].map(min_dict).replace([np.nan],['UKN'])
df_complete.sample(20)

,R_No,Mineral,Vol %,mode,SA_Geo_mode,check,SA_geo_min
362,2014963,Opaques,4,4,TRACE,True,OPQ
522,1842268,Quartz,30,30,MAJOR,True,QZ
474,2014961,Olivine,8,8,MINOR,True,OL
186,2138097,K-feldspar,5,5,MINOR,True,KFS
287,2438823,Clinopyroxene,5,5,MINOR,True,CPX
19,2423472,Zircon,Tr,0,RARE,True,ZRN
388,2134857,Biotite,4,4,TRACE,True,BT
512,1842258,Zircon,Tr,0,RARE,True,ZRN
410,2066660,Zoisite,Tr,0,RARE,True,ZO
84,1842268,Sericite,<1,0,RARE,True,SERC


In [40]:
# Output final tables to Excel
now = datetime.now().strftime("%Y%m%d_%H%M")
SA_geo_mineralogy = df_complete[['R_No','Mineral','Vol %','SA_geo_min','SA_Geo_mode']]

with pd.ExcelWriter(pdf_path+'\\'+now+'_SA_geodata_mineralogy.xlsx') as writer:
    SA_geo_mineralogy.to_excel(writer, sheet_name = 'mapped_mineralogy', index=False)
    df_failed.to_excel(writer, sheet_name = 'failed samples', index=False)
    missing_mins.to_excel(writer, sheet_name = 'missing_minerals')